In [1]:
%cd ..
%reload_ext autoreload
%autoreload 2

D:\ownCloud\Uni\Semester Ma 5\3D Scanning & Motion Capture (IN2354)\Project\3D-FaceReconstruction


In [2]:
import eos
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pyrender
import trimesh
import pickle

from xml.dom import minidom
from collections import defaultdict

from face_reconstruction.model import BaselFaceModel
from face_reconstruction.landmarks import save_bfm_landmarks

from env import BASEL_FACE_MODEL_PATH

# 1. Load model

In [3]:
bfm = BaselFaceModel.from_h5("model2019_face12.h5")

In [4]:
n_shape_coefficients = bfm.get_n_shape_coefficients()
n_expression_coefficients = bfm.get_n_expression_coefficients()
n_color_coefficients = bfm.get_n_color_coefficients()

In [5]:
face_mesh = bfm.draw_sample([0 for _ in range(n_shape_coefficients)], [0 for _ in range(n_expression_coefficients)], [0 for _ in range(n_color_coefficients)])

In [6]:
face_trimesh = bfm.convert_to_trimesh(face_mesh)
#face_trimesh.export(f"{BASEL_FACE_MODEL_PATH}/model2019_face12.ply")

# 2. Load manually picked landmark coordinates from MeshLab file

In [7]:
def parse_xml_point(xml_point):
    point_id = int(xml_point.attributes['name'].value)
    coordinates = np.array([float(xml_point.attributes['x'].value), float(xml_point.attributes['y'].value), float(xml_point.attributes['z'].value)])
    return point_id, coordinates

In [8]:
xml_doc = minidom.parse(f"{BASEL_FACE_MODEL_PATH}/model2019_face12_landmarks_v2.pp")

In [9]:
points = xml_doc.getElementsByTagName("point")
multi_pie_landmark_coordinates = [parse_xml_point(point) for point in points]

In [10]:
multi_pie_landmark_coordinates

[(1, array([-69.931007,  28.560308,  48.61932 ])),
 (2, array([-69.627991 ,   9.4500484,  48.533798 ])),
 (3, array([-65.37912 , -14.477787,  51.418316])),
 (4, array([-61.998131, -33.647957,  52.082237])),
 (5, array([-53.668434, -53.548779,  52.107006])),
 (6, array([-41.075741, -69.155334,  56.70562 ])),
 (7, array([-29.112886, -77.756973,  63.635536])),
 (8, array([-13.517088, -84.987083,  70.500969])),
 (9, array([  1.4427356, -86.341873 ,  73.674377 ])),
 (10, array([ 17.884701, -83.655823,  69.047203])),
 (11, array([ 33.397823, -75.292313,  60.388084])),
 (12, array([ 47.540726, -62.847561,  53.251385])),
 (13, array([ 57.802597, -47.262321,  50.653191])),
 (14, array([ 63.404911, -29.730335,  51.239803])),
 (15, array([ 65.886734, -11.012751,  51.095459])),
 (16, array([69.782455 ,  9.7624693, 48.27858  ])),
 (17, array([68.999817, 29.877851, 49.063202])),
 (18, array([-57.599289,  45.260448,  80.311104])),
 (19, array([-51.050247,  50.861427,  90.475594])),
 (20, array([-43.9

# 3. Find corresponding vertex IDs in mesh

In [11]:
closest_vertices = {}
closest_distances = {}
for vertex_id, vertex in enumerate(face_trimesh.vertices):
    for landmark_idx, landmark in multi_pie_landmark_coordinates:
        distance = np.linalg.norm(landmark - vertex)
        if landmark_idx not in closest_vertices or distance < closest_distances[landmark_idx]:
            closest_vertices[landmark_idx] = vertex_id
            closest_distances[landmark_idx] = distance

In [12]:
closest_vertices

{1: 16506,
 2: 15592,
 3: 21354,
 4: 17304,
 5: 18246,
 6: 18214,
 7: 21562,
 8: 16358,
 9: 15850,
 10: 4233,
 11: 4234,
 12: 4493,
 13: 1865,
 14: 7456,
 15: 197,
 16: 7938,
 17: 34,
 18: 22050,
 19: 25251,
 20: 17458,
 21: 27089,
 22: 21910,
 23: 11856,
 24: 12306,
 25: 66,
 26: 68,
 27: 3635,
 28: 10535,
 29: 15877,
 30: 15925,
 31: 15841,
 32: 15384,
 33: 18137,
 34: 23372,
 35: 13285,
 36: 13632,
 37: 23870,
 38: 23053,
 39: 22520,
 40: 16017,
 41: 23083,
 42: 17646,
 43: 6993,
 44: 10654,
 45: 600,
 46: 9918,
 47: 11115,
 48: 9131,
 49: 22858,
 50: 27020,
 51: 18364,
 52: 7854,
 53: 5617,
 54: 6265,
 55: 5032,
 56: 1727,
 57: 12603,
 58: 25911,
 59: 19752,
 60: 19127,
 61: 26201,
 62: 21620,
 63: 18197,
 64: 10941,
 65: 1739,
 66: 4657,
 67: 18295,
 68: 26195}

In [13]:
save_bfm_landmarks(closest_vertices, "model2019_face12_landmarks_v2")

# 4. Visualize outputs

In [14]:
for vertex_id in closest_vertices.values():
    vertex = face_trimesh.visual.vertex_colors[vertex_id]
    vertex[0] = 255
    vertex[1] = 0
    vertex[2] = 0

In [15]:
perspective_camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.414)
directional_light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=2.0)

In [16]:
scene = pyrender.Scene()
scene.add(pyrender.Mesh.from_trimesh(face_trimesh))
scene.add(perspective_camera, pose=np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 300], [0, 0, 0, 1]])) # Position camera just in front of face
scene.add(directional_light)

In [17]:
pyrender.Viewer(scene, use_raymond_lighting=True)

Viewer(width=640, height=480)